In [8]:
import pandas as pd
import datetime

filepath = '../../candle_data/BINANCE_BTCUSDT, 60.csv'

df = pd.read_csv(filepath, index_col='time',
     usecols=['time', 'open', 'high', 'low', 'close', 'volume'],
     parse_dates=['time'],
     date_parser=lambda epoch: pd.to_datetime(epoch, unit='s')
     )
df

,open,high,low,close,volume
time,,,,,
2018-01-01 00:00:00,13715.65,13715.65,13400.01,13529.01,443.356199
2018-01-01 01:00:00,13528.99,13595.89,13155.38,13203.06,383.697006
2018-01-01 02:00:00,13203.00,13418.43,13200.00,13330.18,429.064572
2018-01-01 03:00:00,13330.26,13611.27,13290.00,13410.03,420.087030
2018-01-01 04:00:00,13434.98,13623.29,13322.15,13601.01,340.807329
...,...,...,...,...,...
2020-01-17 00:00:00,8720.15,8743.99,8672.44,8690.31,1988.852577
2020-01-17 01:00:00,8690.25,8738.80,8677.01,8738.51,1156.787342
2020-01-17 02:00:00,8738.80,8780.00,8718.16,8757.06,2083.041074


In [9]:
import numpy as np
period = 3
fast_period = 2
slow_period = 30
df['direction'] = df['close'].diff(period)
df['directon2'] = df['close'] - df['close'].shift(period)
df['change'] = df.close.diff()
df['volatility'] = df.close.diff().abs().rolling(period).sum()
efficiency_ratio = df['direction'].abs() / df['volatility']
df['period'] = (efficiency_ratio * period).dropna().astype(int)
fast = 2.0 / (fast_period + 1.0)
slow = 2.0 / (slow_period + 1.0)
sc = (efficiency_ratio * (fast - slow) + slow) ** 2.0
hl2 = (df['high'] + df['low']) / 2.0
answer = np.zeros(sc.size)
N = len(answer)
first_value = True
# k = nz(k[1])+(sc*(hl2-nz(k[1])))
for i in range(N):
    if sc[i] != sc[i]:
        answer[i] = np.nan
    else:
        if first_value:
            answer[i] = df['close'][i]
            first_value = False
        else:
            # answer[i] = answer[i - 1] + sc[i] * (price[i] - answer[i - 1])
            answer[i] = answer[i - 1] + sc[i] * (hl2[i] - answer[i - 1])
df['kama'] = answer
df


,open,high,low,close,volume,direction,directon2,change,volatility,period,kama
time,,,,,,,,,,,
2018-01-01 00:00:00,13715.65,13715.65,13400.01,13529.01,443.356199,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 01:00:00,13528.99,13595.89,13155.38,13203.06,383.697006,NaN,NaN,-325.95,NaN,NaN,NaN
2018-01-01 02:00:00,13203.00,13418.43,13200.00,13330.18,429.064572,NaN,NaN,127.12,NaN,NaN,NaN
2018-01-01 03:00:00,13330.26,13611.27,13290.00,13410.03,420.087030,-118.98,-118.98,79.85,532.92,0.0,13410.030000
2018-01-01 04:00:00,13434.98,13623.29,13322.15,13601.01,340.807329,397.95,397.95,190.98,397.95,3.0,13437.892222
...,...,...,...,...,...,...,...,...,...,...,...
2020-01-17 00:00:00,8720.15,8743.99,8672.44,8690.31,1988.852577,-0.84,-0.84,-29.70,59.30,0.0,NaN
2020-01-17 01:00:00,8690.25,8738.80,8677.01,8738.51,1156.787342,18.13,18.13,48.20,78.27,0.0,NaN
2020-01-17 02:00:00,8738.80,8780.00,8718.16,8757.06,2083.041074,37.05,37.05,18.55,96.45,1.0,NaN


In [3]:
price = df['close']
n = 12
pow1, pow2 = 2,30
import numpy as np

absDiffx = abs(price - price.shift(1))
ER_num = abs(price - price.shift(n))
# ER_den = pd.stats.moments.rolling_sum(absDiffx, n)
ER_den = price.diff().abs().rolling(n).sum()
df['er_den'] = ER_den

ER = ER_num / ER_den

sc = (ER * (2.0 / (pow1 + 1) - 2.0 / (pow2 + 1.0)) + 2 / (pow2 + 1.0)) ** 2.0
df['sc'] = sc
df['kk'] = np.zeros(sc.size)
# k = nz(k[1])+(sc*(hl2-nz(k[1])))
hl2 = (df['high'] + df ['low']) / 2

answer = np.zeros(sc.size)
N = len(answer)
first_value = True
# k = nz(k[1])+(sc*(hl2-nz(k[1])))
for i in range(N):
    if sc[i] != sc[i]:
        answer[i] = np.nan
    else:
        if first_value:
            answer[i] = price[i]
            first_value = False
        else:
            # answer[i] = answer[i - 1] + sc[i] * (price[i] - answer[i - 1])
            answer[i] = answer[i - 1] + sc[i] * (hl2[i] - answer[i - 1])
df['answer']=answer
df

,open,high,low,close,volume,er_den,sc,kk,answer
time,,,,,,,,,
2018-01-01 00:00:00,13715.65,13715.65,13400.01,13529.01,443.356199,NaN,NaN,0.0,13529.01
2018-01-01 01:00:00,13528.99,13595.89,13155.38,13203.06,383.697006,NaN,NaN,0.0,NaN
2018-01-01 02:00:00,13203.00,13418.43,13200.00,13330.18,429.064572,NaN,NaN,0.0,NaN
2018-01-01 03:00:00,13330.26,13611.27,13290.00,13410.03,420.087030,NaN,NaN,0.0,NaN
2018-01-01 04:00:00,13434.98,13623.29,13322.15,13601.01,340.807329,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...
2020-01-17 00:00:00,8720.15,8743.99,8672.44,8690.31,1988.852577,303.45,0.015235,0.0,NaN
2020-01-17 01:00:00,8690.25,8738.80,8677.01,8738.51,1156.787342,326.94,0.020770,0.0,NaN
2020-01-17 02:00:00,8738.80,8780.00,8718.16,8757.06,2083.041074,336.38,0.036633,0.0,NaN


In [ ]:
answer = np.zeros(sc.size)
N = len(answer)
first_value = True
# k = nz(k[1])+(sc*(hl2-nz(k[1])))
for i in range(N):
    if sc[i] != sc[i]:
        answer[i] = np.nan
    else:
        if first_value:
            answer[i] = price[i]
            first_value = False
        else:
            answer[i] = answer[i - 1] + sc[i] * (price[i] - answer[i - 1])
df['kama'] = answer

In [64]:
df

,open,high,low,close,volume,er_den,sc,kk,answer,direction,volatility,period
time,,,,,,,,,,,,
2018-01-01 00:00:00,13715.65,13715.65,13400.01,13529.01,443.356199,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 01:00:00,13528.99,13595.89,13155.38,13203.06,383.697006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 02:00:00,13203.00,13418.43,13200.00,13330.18,429.064572,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 03:00:00,13330.26,13611.27,13290.00,13410.03,420.087030,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 04:00:00,13434.98,13623.29,13322.15,13601.01,340.807329,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-17 00:00:00,8720.15,8743.99,8672.44,8690.31,1988.852577,303.45,0.015235,8852.409645,8700.893270,29.69,303.45,1.0
2020-01-17 01:00:00,8690.25,8738.80,8677.01,8738.51,1156.787342,326.94,0.020770,8871.808145,8701.674566,43.22,326.94,1.0
2020-01-17 02:00:00,8738.80,8780.00,8718.16,8757.06,2083.041074,336.38,0.036633,9059.308148,8703.703506,70.88,336.38,2.0


In [66]:
 (df['high'] + df ['low']) / 2

time
2018-01-01 00:00:00    13557.830
2018-01-01 01:00:00    13375.635
2018-01-01 02:00:00    13309.215
2018-01-01 03:00:00    13450.635
2018-01-01 04:00:00    13472.720
                         ...    
2020-01-17 00:00:00     8708.215
2020-01-17 01:00:00     8707.905
2020-01-17 02:00:00     8749.080
2020-01-17 03:00:00     8795.725
2020-01-17 04:00:00     8784.460
Length: 17829, dtype: float64

In [73]:
import pandas as pd
def vperiod(data, period):
    '''
    Formula:
      - direction = close - close_period
      - volatility = sumN(abs(close - close_n), period)
      - efficientcy_ratio = abs(direction / volatility)
      - fast = 2 / (fast_period + 1)
      - slow = 2 / (slow_period + 1)
      - smfactor = squared(efficienty_ratio * (fast - slow) + slow)
      - smfactor1 = 1.0  - smfactor
    '''
    fast_period, slow_period = 2, 30
    data['direction'] = data['close'].diff(period)
    data['volatility'] = data.close.diff().abs().rolling(period).sum()
    efficiency_ratio = data['direction'].abs() / data['volatility']
    data['period'] = (efficiency_ratio * period).dropna().astype(int)
    fast = 2.0 / (fast_period + 1.0)
    slow = 2.0 / (slow_period + 1.0)
    sc = (efficiency_ratio * (fast - slow) + slow) ** 2.0
    hl2 = (data['high'] + data ['low']) / 2
    answer = np.zeros(sc.size)
    N = len(answer)
    first_value = True
    # k = nz(k[1])+(sc*(hl2-nz(k[1])))
    for i in range(N):
        if sc[i] != sc[i]:
            answer[i] = np.nan
        else:
            if first_value:
                answer[i] = df['close'][i]
                first_value = False
            else:
                # answer[i] = answer[i - 1] + sc[i] * (price[i] - answer[i - 1])
                answer[i] = answer[i - 1] + sc[i] * (hl2[i] - answer[i - 1])
    data['kama'] = answer
    
period = 12
vperiod(df, period)

df

,open,high,low,close,volume,er_den,sc,kk,answer,direction,volatility,period,kama
time,,,,,,,,,,,,,
2018-01-01 00:00:00,13715.65,13715.65,13400.01,13529.01,443.356199,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2018-01-01 01:00:00,13528.99,13595.89,13155.38,13203.06,383.697006,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2018-01-01 02:00:00,13203.00,13418.43,13200.00,13330.18,429.064572,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2018-01-01 03:00:00,13330.26,13611.27,13290.00,13410.03,420.087030,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2018-01-01 04:00:00,13434.98,13623.29,13322.15,13601.01,340.807329,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-17 00:00:00,8720.15,8743.99,8672.44,8690.31,1988.852577,303.45,0.015235,0.0,8694.276865,-29.69,303.45,1.0,8694.276865
2020-01-17 01:00:00,8690.25,8738.80,8677.01,8738.51,1156.787342,326.94,0.020770,0.0,8694.559920,43.22,326.94,1.0,8694.559920
2020-01-17 02:00:00,8738.80,8780.00,8718.16,8757.06,2083.041074,336.38,0.036633,0.0,8696.557159,70.88,336.38,2.0,8696.557159


In [84]:
df['close'].iloc[1]==df['close'][0]

False

In [48]:
s = df['close']
s.name

'close'

In [50]:
df['close'].shift(s['period'])

KeyError: 'vperiod'

In [ ]:
def vmomentum(x):
    return df['close'].shift(x['period'])[x.name]

df.dropna(inplace=True)
df['vmomentum'] = df.apply(vmomentum, axis=1)
df

In [ ]:
import numpy as np

df['profit'] = np.where(df.close.shift(1) > df.vmomentum.shift(1), df.close / df.close.shift(1), 1.03 ** (1 / 12)).cumprod()
